<a href="https://colab.research.google.com/github/pietroperrone01/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import xarray as xr

In [5]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


# Part 1: loading data

In [6]:
# this is for getting rid of edges (usually artifacted)
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

# define Svalbard area
svalbard_geom = ee.Geometry.Polygon(
    [[-2.434071, 81.099918],
     [46.394056, 81.099918],
     [46.394056, 74.304052],
     [-2.434071, 74.304052],
     [-2.434071, 81.099918]]
)

In [27]:
# Sentinel-1 collection for Svalbard
svalbard_s1_dict = {}
polarizations = ['HH', 'VV', 'HV', 'VH']

for pol in polarizations:
    svalbard_s1_dict[pol] = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select(pol)
    .map(mask_edge)
    )


In [39]:
svalbard_s1_pol = [
svalbard_s1HH := svalbard_s1_dict['HH'],
svalbard_s1VV := svalbard_s1_dict['VV'],
svalbard_s1HV := svalbard_s1_dict['HV'],
svalbard_s1VH := svalbard_s1_dict['VH']]

In [38]:
# time ranges
spring = ee.Filter.date('2025-03-01', '2025-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-06-10')
summer = ee.Filter.date('2025-06-11', '2025-08-31')

seasons = [spring, late_spring, summer]

In [10]:
Map = geemap.Map()
Map.centerObject(svalbard_geom, zoom=6)
Map.addLayer(svalbard_s1HH, {'min': -25, 'max': 5}, 'Sentinel-1') # display a certain range of dB
Map

Map(center=[78.48364888332274, 21.979992499999994], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
maps = [
    SpringMap := geemap.Map(),
    LateSpringMap := geemap.Map(),
    SummerMap := geemap.Map()
]

for ic in svalbard_s1:
    svalbard_s1HH_spring = ic.filter(spring)

In [71]:
# Building the Sentinel-1 polarizations-seasons dataframe
season_names = ['spring', 'late_spring', 'summer']

svalbard_s1 = pd.DataFrame(
    index=polarizations,
    columns=season_names, # Use descriptive season names as columns
)

for pol in polarizations:
    for i, season_filter in enumerate(seasons):

        filtered_collection = svalbard_s1_dict[pol].filter(season_filter)

        svalbard_s1.loc[pol, season_names[i]] = filtered_collection

display(svalbard_s1)

,spring,late_spring,summer
HH,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."
VV,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."
HV,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."
VH,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."


In [78]:
svalbard_s1_spring = [
svalbard_s1HH_spring := svalbard_s1.loc['HH', 'spring'],
svalbard_s1VV_spring := svalbard_s1.loc['VV', 'spring'],
svalbard_s1HV_spring := svalbard_s1.loc['HV', 'spring'],
svalbard_s1VH_spring := svalbard_s1.loc['VH', 'spring']]

svalbard_s1_latespring = [
svalbard_s1HH_late_spring := svalbard_s1.loc['HH', 'late_spring'],
svalbard_s1VV_late_spring := svalbard_s1.loc['VV', 'late_spring'],
svalbard_s1HV_late_spring := svalbard_s1.loc['HV', 'late_spring'],
svalbard_s1VH_late_spring := svalbard_s1.loc['VH', 'late_spring']]

svalbard_s1_summer = [
svalbard_s1HH_summer := svalbard_s1.loc['HH', 'summer'],
svalbard_s1VV_summer := svalbard_s1.loc['VV', 'summer'],
svalbard_s1HV_summer := svalbard_s1.loc['HV', 'summer'],
svalbard_s1VH_summer := svalbard_s1.loc['VH', 'summer']]

In [81]:
# Spring
SpringMap = geemap.Map()
SpringMap.centerObject(svalbard_geom, zoom=6)

for ic, pol in zip(svalbard_s1_spring, polarizations):
    ic_mosaic = ic.mosaic()
    SpringMap.addLayer(ic_mosaic, {'min': -25, 'max': 5}, f'{pol}')

    ic_date = ic.filterDate(ee.Date('2025-03-01'))

SpringMap

Map(center=[78.48364888332274, 21.979992499999994], controls=(WidgetControl(options=['position', 'transparent_…